# Generating a News and Weather Report using APIs

### Objective
Using Python to chain together several APIs, create a an agentic news editor – modelled on a famous personality – who writes a commentary on the news and weather forecast in your location. The program ends by generating an MP3 file of the bulletin being read out to an audience.

### The Plan
- Task 0: Prepare your work environment

- Task 1: Get news headlines from GNews

- Task 2: Fetch weather forecast from WeatherAPI

- Task 3: Generate the news bulletin text using OpenAI chat completions

- Task 4: Generate an audio file using OpenAI Text-to-Speech 

### Sources
To eventually construct our sound clip, we will pull the requisite data from various sources. Check out their API documentation to know more about how to use them:

1. [GNews API](https://gnews.io/docs/v4?python#introduction) 

2. [WeatherAPI](https://www.weatherapi.com/docs/)

3. [OpenAI chat completions API](https://platform.openai.com/docs/guides/text-generation)

4. [OpenAI text-to-speech API](https://platform.openai.com/docs/guides/text-to-speech)


### Before we begin, some preparation

You will need to create accounts on all the sites listed above. Keep in mind that the OpenAI API isn’t free and needs a credit card to use. Make sure you’ve set up your card by going to "_settings > billing_” on the [OpenAI platform portal](https://platform.openai.com/) before we start. Otherwise, you might miss out on the most exciting part: generating the digest and having it read out loud! Don’t worry, we won’t be racking up a massive bill with OpenAI, if you want to be safe you can set up a “_Monthly spend limit_” so you won’t end up paying more than you want.

Please register an account on the following websites:

- [GNews](https://gnews.io/) – Free plan is sufficient

- [WeatherAPI](https://www.weatherapi.com/) – Free plan is sufficient

- [OpenAI API](https://platform.openai.com/) – Needs a paid plan

### A quick refresher on API concepts

Before we get started let's refresh a few important API concepts. If you've already worked with APIs before, or if you completed the [Introduction to APIs in Python course](https://www.datacamp.com/courses/introduction-to-apis-in-python), none of this should be new. For those of you who have never worked with APIs before or have limited experience, this is a useful refresher and will make following along with the rest of the code-along a lot easier.

<details>
  <summary>What is an API?</summary> 
  <p>

APIs, or Application Programming Interfaces, are like the messengers of the digital world, making sure apps, devices, and services can talk to each other smoothly. They let different systems share data and perform actions without getting tangled in each other’s code. You might not notice it, but APIs are behind almost everything we do digitally. Plug in your electric car, and an API’s handling the data exchange with the charger. Check the weather or get an email on your phone? That’s thanks to APIs pulling and delivering data from online sources to your device.

In essence, APIs make life easier for both developers and users, setting the rules for how apps communicate and what they can share. Whether it’s streaming music, ordering food, or tracking fitness, APIs keep it all connected. They’re one of the most powerful, behind-the-scenes technologies that make our digital world feel effortless, despite being complex and varied under the hood.
   </p>
</details>

<details>
  <summary>REST APIs</summary> 
  <p>

APIs come in many different styles and flavors, but for the purpose of this code-along, we will be focusing on REST APIs. REST APIs let two systems communicate over the internet, using HTTP, the same protocol your browser uses to load webpages. Think of it like typing a website address into your browser: you send a request, and the page loads in response. 

With rest APIs, the system that sends the request is called **the client**, the system the receives the request and returns a response is usually **the server**. 

![Request response cycle](resources/request-response-cycle.png)
   </p>
</details>


<details>
  <summary>HTTP Verbs</summary> 
  <p>

REST APIs are based on HTTP verbs. These describe the action the client wishes to perform on a resource on server. In total there are 9 HTTP verbs, in practice you'll be using the 4 most common verbs described in the table below.

| Verb   | Action |
|-|-|
| `GET`    | Reading a resource |
| `POST`   | Create a new resource |
| `PUT`    | Update an existing resource |
| `DELETE` | Delete a resource |
   </p>
</details>


<details>
  <summary>URLs and parameters</summary> 
  <p>

![Structure of URLs](resources/url-structure.png)

A REST API is accessible over the internet through a URL, which acts like an address that points to a specific resource on the server. This URL includes several parts, as shown in the image: the protocol (http://), the server’s domain (350.5th-ave.com), the port number (like :80), the path to the specific resource (e.g., /unit/243), and optional query parameters (e.g., ?floor=77).

When interacting with REST APIs, you’ll often need to include additional information, like filters or specific search terms, to get exactly the data you want. This extra information is usually passed as query parameters in the URL. These parameters help refine your request to retrieve more specific data, making the API flexible and responsive to a wide range of needs.
   </p>
</details>


<details>
  <summary>Headers</summary> 
  <p>

Headers carry additional information about the request or response, providing context for how the message should be processed. They can specify how the authenticate or what the content type is that's being sent. Headers essentially help the server and client understand the message’s format and how to handle it. Each header is a key-value pair, written as `Key: Value`, where the key is case-insensitive.
   </p>
</details>


<details>
  <summary>Response status codes</summary> 
  <p>

There are over 70 HTTP status codes, grouped into five main categories. These categories help us quickly understand the nature of the response. Codes starting with 1XX are informational, 2XX indicate success, 3XX handle redirections, 4XX signal client-side errors, and 5XX point to server-side issues.

Among these, three key codes to remember are:
- `200` **OK**: The request was successful, and the server returned the expected data.
- `404` **Not Found**: The server couldn’t find the requested resource.
- `500` **Internal Server Error**: Something went wrong on the server’s side, causing the request to fail.

   </p>
</details>

## Task 0: Prepare your work environment

### Packages used

For this code-along we will be using the `os` and `requests` packages.

_Tip: You can find the documentation of the `requests` package online at https://requests.readthedocs.io/_

### Storing secrets

All three of the APIs we plan on using support API-key based authentication so let's set up these API keys as environment variables for our workbook. We should not copy-paste these API-keys directly into our python code because that risks accidentally leaking them when you for example would copy the workbook. Given some of these APIs incur a cost when using them, it's of utmost importance we store them securely.

I store my API keys in a `.env` file in my local drive and promptly add it to the `.gitignore` file before pushing it to the hub. That way, my program is able to refer to my API keys. You should do the same for your copy. Just make sure that you name the variables as described below.

1. Create three environment variables with the following names

    - `API_KEY_GNEWS`

    - `API_KEY_WEATHERAPI`

    - `API_KEY_OPENAI`

2. Save the environment variables in a `.env` file in your local drive.

3. Load all API keys into Python variables using the `os` package

### Loading secrets

Next we need to import the environment variables into our Python workbook using the `os` package. This will make the API keys stored as environment variables available as within our Python code. This way, we can securely use our API keys without adding them to our source code directly.

- Import the `os` package.

- Load the values from the environment variables into Python variables using the `os` package.

In [1]:
# Import the python-dotenv package if not already installed
!pip install python-dotenv

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

# Load the environment variables we've set up into Python variables
api_key_openai = os.environ['API_KEY_OPENAI']
api_key_gnews = os.environ['API_KEY_GNEWS']
api_key_weatherapi = os.environ['API_KEY_WEATHERAPI']

# Check if we've successfully loaded the environment variables
print("OpenAI API key", "loaded" if bool(api_key_openai) else "loading failed...")
print("GNews API key", "loaded" if bool(api_key_gnews) else "loading failed...")
print("WeatherAPI key", "loaded" if bool(api_key_weatherapi) else "loading failed...")

OpenAI API key loaded
GNews API key loaded
WeatherAPI key loaded


## Task 1: Get news headlines from GNews

In this task we'll use the gnews.io API to fetch the most recent news headlines. 

Let's first start by importing the `requests` library. We only need to do this once. Any cells further down this notebook will have access to `requests` once we've imported it.

In [3]:
# Import the requests libraryimport requests
import requests

Now let's consult the [gnews.io documentation](https://gnews.io/docs/v4?python#authentication). We'll be using the _top\_headlines_ API endpoint. This API has a few required URL parameters, which means we'll need to include these in our API call, otherwise we'll get an error!

The gnews.io API requires authentication by sending along an API key along with every request made. The documentation shows us that we can send this API key as a URL parameter called `apiKey`.

- Create a new Python dictionary called `gnews_params` with all the required parameters for our API request URL.

In [4]:
# Create a dictionary to collate the parameters that get sent when making the API call.
gnews_params = {
    'apikey': f"{api_key_gnews}",
    'q': '',        # The search query string is important. We'll fill it before making the request.
    'category': 'general',
    'lang': 'en',
    'country': 'in',
    'max': 10,
    'in': 'title,description',
    'from': '',
    'to': '',
    'sortby': 'relevance'
}

Ensure you use the correct key/value combination for the API key in the `gnews_params` dictionary. Refer to the [gnews.io API docs](https://gnews.io/docs/v4?python) to know what mandatory and optional parameters are expected to be passed.

Now we've got everything ready, let's fire the API request and save the data we need to a variable we'll use later.

- Send the API request to the `https://gnews.io/api/v4/top-headlines` endpoint.

- Evaluate if the response is successful.

- Create a new list called `headline_articles`

In [5]:
# GNews offers two endpoints.
gnews_topheadlines = "https://gnews.io/api/v4/top-headlines"
gnews_search = "https://gnews.io/api/v4/search"

# Customise the search by setting parameters here
gnews_params['q'] = 'india'
gnews_params['lang'] = 'en'

# Send a GET request to the top-headlines API
response = requests.get(gnews_topheadlines, params=gnews_params)

# Raise a SystemExit exception when we don't receive the expected response status_code
if response.status_code == 200:
    print(f"Response Code: {response.status_code}. Please proceed.")
else:
    raise SystemExit('Something went wrong, please check the response text')
    print(f"Response Code: {response.text}. Please proceed.")

Response Code: 200. Please proceed.


-  Raising a `SystemExit` stops the notebook's execution, preventing any subsequent cells that depend on this one from raising errors.

-  A status code of `200` means the server successfully processed the request. Status codes in the `400`-`499` range indicate an error.

-  The `json` library provides tools to work between string data received and the JSON data format.

In [7]:
# Get the JSON content from the response
import json
headline_articles = response.json()

# Create a new list `headline_articles` with only the title and description of each article
print(f"Total articles received: {headline_articles['totalArticles']}")
print(f"First article:\n{headline_articles['articles'][0]}")

Total articles received: 62446
First article:
{'title': "Exit poll highlights: Mahayuti likely to retain Maharashtra, close fight in J'khand", 'description': 'Exit poll result highlights: Projections by Peoples Pulse and Matrize predict a comfortable win for Mahayuti in Maharashtra and NDA in Jharkhand. The exit poll by News24-Chanakya, however, predicts a close contest between MVA and Mahayuti in Maharashtra.', 'content': 'Exit poll result live: In contrast to the majority of the pollsters, Axis My India projected a massive victory for the JMM-Congress alliance.\nExit poll result highlights: Thousands of voters cast their ballots on Wednesday to elect new state governme... [1641 chars]', 'url': 'https://www.hindustantimes.com/india-news/exit-polls-2024-live-litmus-test-for-bjp-ncp-sena-and-sorens-101732092593271.html', 'image': 'https://www.hindustantimes.com/ht-img/img/2024/11/20/550x309/Jharkhand-chief-minister-Hemant-Soren-and-his-MLA-_1725362824563_1732112366950.jpg', 'publishedAt

## Task 2: Fetch weather forecast from WeatherAPI

Now we will use the `requests` library to perform an API call to WeatherAPI.com.

Reading the [weatherapi.com/docs](https://www.weatherapi.com/docs/), we learn that we need to use the `http://api.weatherapi.com/v1` base url followed by one of several endpoint options depending on what we want. For example:

- current (`/current.json`)

- forecast (`/forecast.json`)

- search (`/search.json`)

- history (`/history.json`)

- future (`/future.json`)

The documentation also notes that the `key` (API key) and `q` (search string) are mandatory parameters for all requests, and specifies several formats for valid search strings. Now would be a good time to read through the docs.

As before, we need to create a dictionary with all the parameters expected of our API call. Let's call `/current.json` to get the current weather forecast for our location.

In [8]:
weatherapi_endpoint = "http://api.weatherapi.com/v1"

current = "/current.json"
forecast = "/forecast.json"
search = "/search.json"
history	= "/history.json"
alerts = "/alerts.json"
marine = "/marine.json"
future = "/future.json"
timezone = "/timezone.json"
sports = "/sports.json"
astronomy = "/astronomy.json"
ip_lookup = "/ip.json"

# Create a dictionary with headers for the request
weatherapi_params = {
    'key': f"{api_key_weatherapi}",
    'q': '',
    'aqi': 'yes',
    'days': 0,
}

In [25]:
# Send a GET request to the `/find_places` API to find the `place_id` for our location
weatherapi_params['q'] = 'delhi india'

response = requests.get(weatherapi_endpoint+current, params=weatherapi_params)

# Raise a SystemExit exception when we don't receive the expected response status_code
if response.status_code == 200:
    print(f"Response Code: {response.status_code}. Please proceed.")
    weather_current = response.json()
else:
    raise SystemExit('Something went wrong, please check the response text')
    print(f"Response Code: {response.text}. Please proceed.")

Response Code: 200. Please proceed.


In [26]:
# Extract the place_id from the response
our_location = weather_current['location']
print(f"Location: {our_location['name']}")

weather_current = weather_current['current']
print(f"PM 2.5 Level: {weather_current['air_quality']['pm2_5']:.0f}")

Location: Delhi
PM 2.5 Level: 243


While we're here, let's make another request, this time to the `/forecast.json` endpoint, to see what the weather is going to be like the next few days.

In [27]:
weatherapi_params['days'] = 2

response = requests.get(weatherapi_endpoint+forecast, params=weatherapi_params)

if response.status_code == 200:
    print(f"Response Code: {response.status_code}. Please proceed.")
else:
    raise SystemExit('Something went wrong, please check the response text')
    print(f"Response Code: {response.text}. Please proceed.")

Response Code: 200. Please proceed.


In [28]:
weather_forecast = response.json()['forecast']['forecastday']
print(f"Forecast available for the following dates:\n{[_['date'] for _ in weather_forecast]}")

weather_forecast[1]['day']['air_quality']     # Tomorrow's weather

Forecast available for the following dates:
['2024-11-21', '2024-11-22']


{'co': 2977.960416666667,
 'no2': 152.95645833333333,
 'o3': 62.541666666666664,
 'so2': 102.22020833333333,
 'pm2_5': 222.670625,
 'pm10': 225.630625,
 'us-epa-index': 5,
 'gb-defra-index': 10}

## Task 3: Generate the news bulletin text using OpenAI chat completions

Now that we've gathered all data for our news update, we can now generate a message using AI. We'll do this by sending a prompt to the **OpenAI chat completions API** to generate this message. But first let's use the data we have collected to construct the prompt.

### Creating the AI prompt

Writing a system and user prompt is out of scope of this code-along, so the prompts have already been created. We just need to put them into Python variables and add the data we received from the GNews and WeatherAPI.

Our prompt will consist of two messages. First is the _system message_, it contains instructions for the model, and typically describes what the model is supposed to do and how it should generally behave and respond.

- Create a variable `system_message` with the system message

In [29]:
# Create the system_message variable
system_message = '''
You are a news editor tasked with writing up a news bulletin meant to be read out to an audience 
that wants to stay informed and also enjoys intellectually engaging humour. 
The update should be about 2-5 minutes long, incorporating today's weather, tomorrow's forecast, and the top 10 news headlines 
in a manner that might be spoken by a famous person (such as Salman Rushdie, Mahatma Gandhi, etc.) 
Do not output anything else than the text, don't include any markup, lists, or other structural elements. 
The text will be sent to a text-to-speech API to generate an MP3, so make sure the output contains nothing that should not be read out loud.

Structure the monologue as follows:
1. Greeting: Introduce yourself in a by-the-way manner without revealing your character's name to the audience at any point.
2. Weather Summary: Describe the day’s weather and what can be expected in tomorrow's forecast, as our chosen character would say it. 
Lavish attention on the air quality situation.
3. News Headlines: Present every news item available along with a sardonic take on the event in the imagined words of our chosen character.
4. Closing: Wrap up with a concluding remark that leaves the reader with a smile, positive thought, or playful nudge.
Be creative in how you incorporate the tone and style, ensuring that the text is engaging and enjoyable to listen to.
'''

Next is the _user message_, it contains a request with instructions to the model. Think of this as the message you would send to ChatGPT.

- Create a variable `user_message` with the user message

- Add the variables with the forecast and headlines we created before into the `user_message` variable

In [30]:
# Create the user_message variable
newsreader = 'SALMAN RUSHDIE' 

user_message = f'''
Please generate a light-hearted News and Weather Bulletin.

We are currently at {our_location}. 
This is what the air is like today: {weather_current['air_quality']}")

Here's the weather forecast for today and tomorrow: {weather_forecast}

Lastly, here are the latest news headlines in JSON format: 
{headline_articles['articles']}

Generate the text as specified in the system prompt, following the structure of greeting, weather summary, 10 headlines, and a closing remark.

Today, our news reader is {newsreader}!
'''

### Prompting the AI engine

Now for the exciting part, let's use the OpenAI chat completions API to generate our message! 

Reading [the OpenAI chat completions documentation](https://platform.openai.com/docs/api-reference/chat/create), the first thing we notice is that we'll need to use another HTTP verb: `POST`. Previously we only had to "get" data from the API, but now we are going to be sending data (the prompt) _to_ the API to create a new response. We'll actually be creating something new here, hence we need to use the `post()` function of the `requests` library.

This API also requires authentication, specifically _Bearer authentication_. This means we'll need to add an `authorization` header.

**Note:** OpenAI actually offers a really easy to use [Python package called `openai`](https://platform.openai.com/docs/libraries#python-library) that makes using the API a lot easier, but for the purpose of learning the underlying concepts of APIs, we'll use the `requests` package and create the API requests ourself.

- Create a dictionary called `openai_headers`

- Add the correct Bearer authorization header to the dictionary

In [31]:
# Create a dictionary with headers for the request
openai_headers = {
    "authorization": f"Bearer {api_key_openai}"
}

According to the API docs, there are 2 required elements in the request body: 

- `model`: We'll use the `gpt-4o-mini` model as it's the best model in terms of cost vs quality.

- `messages`: This will be a list in which each entry is a dictionary with two properties, `role` and `content`. For our use-case we'll add two messages: one with the `system` role and one with the `user` role, as created in the previous task.

Create a new dictionary called `completions_request_data`.

In [32]:
# Create a new dictionary with the data we'll be sending to the API
completions_request_data = {
    'model' : "gpt-4o-mini",
    'messages' : [
        {"role":"system","content":system_message},
        {"role":"user","content":user_message}
    ]
}

- Send the API request to the `/chat/completions` API.

- Check if the response is successful and it contains a list of choices. Inspect the structure of the choice object.

- Store the message content of the first choice in the `news_bulletin` variable.

- The OpenAI chat completions API requires a `POST` request to be sent, the `requests` Python package has functions for each of the HTTP verbs, such as `requests.get()` or `requests.put()`.

- `POST` requests require a "payload" of data to be sent to the API. The function used for `POST` requests from `requests` library accepts a `json` argument which allows you to send a serialized JSON object to the API.

In [33]:
# Send a POST request to the `/chat/completions` API with `completions_request_data` as json payload
response = requests.post("https://api.openai.com/v1/chat/completions", 
                          json=completions_request_data,
                          headers=openai_headers)

# Evaluate the response status_code
if response.status_code == 200:
    # Get the JSON content from the response
    response_json = response.json()
    
    # Extract the message content into a variable
    if len(response_json['choices']) > 0:
        news_bulletin = response_json['choices'][0]['message']['content']
    else:
        raise SystemExit('Could not find any place_id for the location')
    
# Raise a SystemExit exception when we don't receive the expected response code.
else:
    raise SystemExit(f'An error occured generating a message ({response.status_code}, {response.text})')

print(news_bulletin)

Hello there, listener! You may be surprised to hear, it's me, Salman Rushdie, stepping away from my rather controversial novelesque existence to bring you not merely news, but a tapestry of poetic absurdity we call a bulletin.

Let's start with the weather—a subject often warmer than the debates surrounding my latest book. Today, in Delhi, we bask in the glow of sunshine, flirting with temperatures hitting 27.1 degrees Celsius, an ideal climate for existential pondering. However, while the sun smiles, the air quality might leave you pondering your life choices. With a US-EPA index of 5 and a DEFRA index of 10, let’s just say it's not quite the fresh breath of enlightenment we seek. 

Tomorrow, the relief is seemingly on the forecast horizon: not a drop of rain, and temperatures will flirt with an even more pleasant 27.8 degrees. But beware! The air persists in its pollution artistry, as smog becomes the canvas for your respiratory endeavors. Ah, Delhi, where the air quality is as sopor

## Task 4: Generate an audio file using OpenAI Text-to-Speech 

With the message generated by OpenAI chat completions, we are now ready to create a spoken version of the text and save it as an MP3 for listening. OpenAI offers an excellent text-to-speech API that features a variety of natural-sounding voices for generating the audio file. Start by reviewing the docs for the [OpenAI TTS (text-to-speech) model](https://platform.openai.com/docs/guides/text-to-speech).

Examining the request body, we find there are three required arguments: 

- For the `model` argument we'll choose the `tts-1` model. 

- You can choose any of the available options for the `voice` argument: *alloy*, *echo*, *fable*, *onyx*, *nova*, and *shimmer*. 

- Finally, the text to be generated should be included in the `input` argument.

Now, let's create a dictionary and send it to the API using the `json` function argument. We will name it `tts_request_data`.

In [34]:
# Create a new dictionary with the data we'll be sending to the text-to-speech API
tts_request_data = {
    'model': 'tts-1',
    'input': news_bulletin,
    'voice': 'shimmer',     # Choose a voice: 'alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer'
}

Now let's send the request, but this time we need to evaluate the response a bit differently. 

This response will not contain readable text; instead, it will provide data for an MP3 audio file. We need to store the data returned by the API in a file on disk. Fortunately, Python includes the necessary file manipulation functions, such as `open()`, which allow us to do this. Next to the filename, the `open()` function also requires a `mode` argument. By passing the `wb` value to this argument, we can open the file for writing binary data.

- Send the API request to the `/audio/speech` API.

- Evaluate if the response is successful.

- Write the response content to a file with the extension `.mp3` file.

In [35]:
# Send a POST request to the `/audio/speech` API with `tts_request_data` as json payload
response = requests.post("https://api.openai.com/v1/audio/speech", 
                        json=tts_request_data,
                        headers=openai_headers)

# Evaluate the response status_code
if response.status_code == 200:

    # Write the API response to a file
    file = open('NewsByTheBullet.mp3', 'wb')
    file.write(response.content)
    file.close()
    
    # Play a "ding" sound when done
    os.system('afplay /System/Library/Sounds/Glass.aiff')  # macOS built-in sound

# Raise a SystemExit exception when we don't receive the expected response code.
# Make sure to include the response status_code and text in the error 
# message to make debugging easier.
else:
    raise SystemExit(f'An error occured generating the audio file ({response.status_code}, {response.text})')

- You don't need any specialized libraries to work with files. Opening a file for writing, writing the contents and closing the file is all possible with the built-in file-related functions of Python.

- Use the `open()` fuction to create a new file object

- Use the correct `mode` argument to the `open()` function so we can write binary data to the file.

- The response content can be accessed using the `response.content` property

- Don't forget to close the file


# What's next?

This is where the code-along ends, but there is an unlimited amount of ways you can further tweak and customize the morning update you've just built. 

A great starting point is including extra data in your morning update. Here are a couple of idea's.

1. Tweak the AI prompts to add a motivational message at the end.
2. Use the Google Calendar API to include an overview of your meetings
3. Use the Gmail API to let you know how many unread emails you have
4. and so on!

[The 'public apis' Github repository](https://github.com/public-apis/public-apis), curated by APILayer, contains a huge list of free APIs you can use to play around with! 